# Species - Plant Resources of South East Asia

In [12]:
from bs4 import BeautifulSoup
import requests
import json,codecs
import pandas as pd

# Step 1: Open and read the HTML file
html_file_path = "data/SEA-species.html"  # Replace with your HTML file path
with open(html_file_path, "r", encoding="utf-8") as file:
    html_content = file.read()

# Step 2: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find the <main> tag
# main_tag = soup.find("main")

# Find the <div> tag with class "container"
container_divs = soup.find_all("article", class_="species-card")

# Find all <a> tags
a_tags = []
data = []
for container_div in container_divs:
    # a_tags += container_div.find_all("a")
    try:
        
        labels = [x.text.lower().strip() for x in container_div.find_all("div", class_="pn-labeled-data-label")]
        contents = [x.text.lower().strip() for x in container_div.find_all("div", class_="pn-labeled-data-content")]
        merged_dict = dict(zip(labels, contents))
        original = {
            "common name":"",
            "genus":"",
            "family":"",
        }
        merged_dict = original | merged_dict  # Merge dicts (dict2 values overwrite dict1)
        card = {
            "species-name" : container_div.find("h1").text.strip(),
            "common-name" : merged_dict["common name"],
            "genus" : merged_dict["genus"],
            "family" : merged_dict["family"],
            "images" : [ x.get("src") for x in container_div.find_all("img")],
            "num-images" : container_div.find_all("span")[0].text.replace(",", "").strip(),
            "num-observations" : container_div.find_all("span")[1].text.replace(",", "").split(" ")[0].strip(),
        }
        data.append(card);
    except Exception as e:
        print(f"An error occurred: {e}")
        print(container_div.find("h1").text.strip())
    
    # break;



filename = "data/SEA-species.json"
with codecs.open(filename, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)
    print(f"Write file {filename}")

Write file data/SEA-species.json


In [7]:
int("1,500")

ValueError: invalid literal for int() with base 10: '1,500'